In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils, models
from torchsummary import summary
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
model = models.resnet18(pretrained=True).to(device)

# Freezing the base model layers to prevent retraining
for param in model.parameters():
    param.requires_grad = False

Notes on the Resnet Model:

** convert images to b&w
- Requires input images dimensions (256,256) ** resize our images
- My additional fully connected layer needs dimensions (2048,10) - 10 for the 10 classes for the 10 style types (--)
- Image preprocessing requires:
  1. (224,224) center crop
  2. image is normalized with mean = 255*[0.485, 0.456, 0.406] and
  std = 255*[0.229, 0.224, 0.225]
  3. transpose it from HWC to CHW layout
- Post-processing involves calculating the softmax probability scores for each class

In [ ]:
classes = 10
model.fc = torch.nn.Linear(512, classes).to(device)
print(model)

In [ ]:
loss_fn = torch.nn.NLLLoss # multi-class classification model loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 

In [ ]:
class MyDataset(Dataset):
    def __init__(self,
                 csv_file,      # images could be provided with in a series of directories
                 root_dir,     # images could be provided as a list as well
                 transform = transforms.ToTensor()):  # provide transformation to apply to each image
      """
      Organize the images and the associated labels into two lists.  Potentially create additional
      lists if more complicated information is need.  Important note: images are NOT
      read and stored in this initializer.  They are read in __getitem__ as needed.
      """
      self.csv_file = csv_file # path of csv file
      self.root_dir = root_dir # directory the photos are in
      self.images = pd.read_csv(self.csv_file)
      # Record the transform that may need to be applied.
      self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        '''
        Return a tuple with the data, ground truth label, and any other data
        associated with a single image.
        '''
        img_name = self.images.iloc[idx, 0] # name of image in 1st column
        img_path = os.path.join(self.root_dir, img_name)
        im = Image.open(img_name)

        if self.transform is not None:
            im = self.transform(im)

        """
        label encodes season
        season = {
            0: 'spring'
            1: 'summer'
            2: 'fall'
            3: 'winter'  
        }
        """
        label = self.images.iloc[idx, 1]
        label = torch.tensor(label, dtype=torch.float32)

        return im, label

In [ ]:
image_transforms = transforms.Compose([transforms.Grayscale(num_output_channels=1), \
                                      transforms.ToTensor(), transforms.Resize((224, 224)), \
                                       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dataset = MyDataset(csv_file='C:\Users\cardon\OneDrive - Rensselaer Polytechnic Institute\SEM8\RCOS\fashion-forecast\SFS_metadata_new.csv',
                    root_dir='C:\Users\cardon\OneDrive - Rensselaer Polytechnic Institute\SEM8\RCOS\crop-data',
                    transform=image_transforms)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.7, 0.15, 0.15], generator=torch.Generator())